### 가상 환경 활성화

1. 원하는 폴더로 이동  
```cd C:\Users\user-name\your-directory-path```
  
<br>  

2. 가상환경 생성  
```python -m venv huggingface-env```
  
<br>  

3. 가상환경 활성화   
```.\huggingface-env\Scripts\activate```

<br>  
>  활성화되면 프롬프트가 (huggingface-env)로 바뀝니다.

<br>  

+) 가상환경 비활성화:  
```deactivate```

### requirements.txt. 설치

가상 환경이 활성화가 되었으면 다음 명령어로 모듈 설치:  

```pip install -r requirements.txt```


만약, 에러가 발생한다면 pip 최신화 먼저 실행:  
```python -m pip install --upgrade pip```

## Hugging Face 간단한 예제

1. Hugging Face와 Gradio를 이용해서 간단한 질의응답을 할 수 있는 간단한 웹앱을 실행해보세요.

1. transformer: BERE, GPT, TS 등 유명한 모델을 사용할 수 있음
2. 사전학습 모델 허브: 연구자들이 학습시킨 모델을 공유하눈 플랫폼
3. 손쉬운 테스트: 파이썬 코드로 대형 AI 모델을 사용가능
4. 다양한 테스크 지원: 거의 모든 NLP 작업 지원

In [ ]:
from transformers import pipeline

sentiment = pipeline("sentiment=analysis")

In [3]:
import gradio as gr
from transformers import pipeline

qa = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", force_download=True)
result = qa(question="What is the capital of France?", context="Paris is the capital of France")
print(result)

/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'score': 0.9976332783699036, 'start': 0, 'end': 5, 'answer': 'Paris'}


In [6]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", force_download=True)

def answer_question(context, question):
    result = qa_pipeline({
        'context' : context,
        'question' : question
    })
    return result['answer']

interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=5, label="Context (문맥 입력)", placeholder="여기에 문맥을 입력하세요."),
        gr.Textbox(lines=1, label="Question (질문)", placeholder="질문을 입력하세요.")
    ],
    outputs=gr.Textbox(label="Answer (답변)"),
    title="🤗 Hugging Face Q&A 데모",
    description="Hugging Face의 Transformers 모델을 이용한 문맥 기반 질의응답 웹앱"
)
interface.launch()

/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


## DDPG 다차원 공간 학습 및 시뮬레이션

1. **BipedalWalker-v3**는 연속 행동 공간 중 대표적인 복잡한 환경입니다.

| 항목               | Pendulum-v1  | BipedalWalker-v3 |
| ---------------- | ------------ | ---------------- |
| 상태 공간 (`state`)  | 3차원          | 24차원             |
| 행동 공간 (`action`) | 1차원 (1개 토크)  | 4차원 (4개 다리 제어)   |
| 보상 구조            | 수직 유지를 위한 보상 | 목표까지 걷는 성과 기반    |
| 종료 조건            | 없음           | 넘어진 경우 조기 종료 가능  |


2. 학습을 위한 라이브러리들을 import 하겠습니다.

In [8]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque
import random

### 환경 불러오기

1. BipedalWalker-v3 환경을 불러온뒤, 학습에 필요한 차원들을 정의하는 코드를 작성해보세요.

In [9]:
env = gym.make("BipedalWalker-v3")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 모델 구조 작성

1. BipedalWalker-v3 환경에 적합한 Actor 신경망을 PyTorch로 정의하세요.  

(힌트: 상태 → 행동 출력 / 연속 공간)

In [10]:
class Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.out = nn.Linear(300, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return torch.tanh(self.out(x)) * max_action

2. Critic 네트워크를 정의하세요. 입력은 상태와 행동을 받아 Q-value를 예측해야 합니다.

In [11]:
class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.out = nn.Linear(300, 1)

    def forward(self, x, u):
        x = torch.cat([x, u], 1) # 1 == dim == 열방향 == 가로로
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return self.out(x)


### Replay Buffer 구현

1. 다음 조건을 만족하는 ReplayBuffer 클래스를 직접 작성하세요.

- 경험 데이터를 저장할 수 있어야 함

- 저장된 경험에서 무작위로 일정량을 샘플링할 수 있어야 함

- 저장된 전체 데이터의 개수를 반환할 수 있어야 함

In [12]:
class ReplayBuffer:
    def __init__(self, size = 1000000):
        self.buffer = deque(maxlen=size)
    def add(self, transition):
        self.buffer.append(transition)  # (state, action, reward, next_state, done) = (s, a, r, s', True)
    def sample(self, batch_size):
        samples = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*samples))
        return (
            torch.FloatTensor(state).to(device),
            torch.FloatTensor(action).to(device),
            torch.FloatTensor(reward).unsqueeze(1).to(device),
            torch.FloatTensor(next_state).to(device),
            torch.FloatTensor(done).unsqueeze(1).to(device)
        )
    
    def __len__(self):
        return len(self.buffer)

### DDPG 신경망과 하이퍼파라미터 초기화

1. DDPG에서 신경망과 하이퍼파라미터를 초기화하는 코드를 작성하세요.  
다음 조건을 만족해야 합니다:

- Actor, Critic, Target Actor, Target Critic 네트워크를 정의하고 GPU 또는 CPU에 할당하세요.

- Target 네트워크는 각각 학습 초기 네트워크의 파라미터로 동기화되어야 합니다.

- Actor와 Critic에 대해 각각 다른 학습률을 갖는 Adam 옵티마이저를 생성하세요.

- 경험을 저장할 수 있는 ReplayBuffer 인스턴스를 생성하세요.

- 할인율(gamma), 타겟 소프트 업데이트 계수(tau), 배치 크기(batch_size), 탐험 노이즈(exploration_noise)를 정의하세요.

In [13]:

actor = Actor().to(device)
critic = Critic().to(device)
target_actor = Actor().to(device)
target_critic = Critic().to(device)
target_actor.load_state_dict(actor.state_dict())
target_critic.load_state_dict(critic.state_dict())
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)
replay_buffer = ReplayBuffer()
gamma = 0.99
tau = 0.005
batch_size = 64
exploration_noise = 0.1

2. DDPG에서 Actor와 Critic을 학습시키는 train() 함수를 작성하세요.  

다음 조건을 충족해야 합니다:

- Replay Buffer에서 일정 수 이상의 데이터가 쌓였을 때만 학습이 시작되도록 하세요.

- Critic 네트워크는 타겟 Q값을 사용해 MSE Loss를 기준으로 학습하세요.

- Actor 네트워크는 Critic이 높게 평가하는 방향으로 정책을 업데이트하세요.

- 학습 후, **타겟 네트워크(actor/critic)** 를 Soft Update 방식으로 업데이트하세요.

- Critic Loss는 ```old_critic_losses``` 리스트에, Episode Reward는 외부에서 ```old_episode_rewards```에 저장된다고 가정합니다.

In [14]:
episode_rewards = []
critic_losses = []
def train():
    if len(replay_buffer) < batch_size:
        return
    
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    with torch.no_grad():
        target_action = target_actor(next_state)
        target_q = reward + (1 - done) * gamma * target_critic(next_state, target_action)
    currnet_q = critic(state, action)
    critic_loss = F.mse_loss(currnet_q, target_q)
    critic_optimizer.zero_grad()
    critic_loss.backward()
    critic_optimizer.step()
    critic_losses.append(critic_loss.item())
    actor_loss = -critic(state, actor(state)).mean()
    actor_optimizer.zero_grad()
    actor_loss.backward()
    actor_optimizer.step()

    for param, target_param in zip(critic.parameters(), target_critic.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
    for param, target_param in zip(actor.parameters(), target_actor.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

3. DDPG 학습 루프를 작성하고 학습을 진행해보세요.

각 에피소드마다 얻은 보상을 누적하여 ```old_episode_reward```로 저장하고, 이를 ```old_episode_rewards``` 리스트에 추가하세요.

3. DDPG 학습 결과를 시각화하고 모델을 저장하세요. 

그래프의 제목은 각각 "Episode Rewards", "Critic Losses"로 지정하세요.

### 하이퍼파라미터 조정

1. 학습 결과 보상이 ```-100 ~ 0``` 사이에 있습니다. 유의미한 학습이 되지 않았다는 뜻으로 하이퍼파라미터를 조정하여 다시 학습해보겠습니다.  

<br>

**noise**는 ```0.1 -> 0.2```로 증가, **episode**는 ```1000~2000``` 사이로 조정, **timestep**은 ```1600``` 이하로 설정하겠습니다.

2. 변경된 하이퍼파라미터를 적용하여 다시 학습을 진행해보겠습니다.  

- 학습을 진행하는 도중 최고 성능의 모델을 저장하세요.

- 하이퍼파라미터를 저장하기 이전의 성능 그래프와 비교해보세요.

3. 최고 성능 모델을 사용하여 시뮬레이션을 진행해보세요.